In [ ]:
import gym
import random
import numpy as np
from time import sleep
from IPython.display import clear_output

# Método de Exibir o estado atual

In [ ]:
def exibir(*args):
    clear_output(wait=True)
    if env is not None:
        env.render() # Renderiza o estado atual do ambiente

    if len(args) > 0:
        for arg in args:
            print(arg, end=" ")
        print("")
    sleep(0.3)

## Criação da Tabela de Valores

Inicializar a tabela-Q preenchida com zeros.

Para as ações seguintes, há uma chance do agente tomá-las de forma aleatória ou seguir a política da tabela Estados x Ações.
Após episódios suficientes, é esperado que o agente atinja uma política ótima e a tabela Estados x Ações esteja com os valores o mais próximo do ideal possível.

In [ ]:
env = gym.make("Taxi-v3")
env.reset() # Reseta o ambiente e retorna um estado inicial aleatório
tabela_q = np.zeros([env.observation_space.n, env.action_space.n]) #  A tabela recebe o espaço de ações e espaço de estados, é iniciada com [0,0]

# Treinamento do Algoritmo

α: quanto mais próximo do fim do treino, com uma base de conhecimento sobre o ambiente já consolidada, uma alta taxa de aprendizado torna-se menos relevante.

γ: Quanto mais próximo do final do treinamento, o comportamento do agente para os finais dos episódios já estão bem definido e, se estiver aprendendo de forma correta, estará de acordo com o esperado. É interessante então melhorar o comportamento para as situações mais iniciais.

ε: Quanto mais atualizada a tabela-q, menos o agente precisa explorar ações aleatórias.


In [ ]:
# variáveis de controle
alpha = 0.1 # Taxa de aprendizado
gamma = 0.6 # Taxa de desconto
epsilon = 0.1 # Chance de selecionar uma ação aleatória em vez de maximizar o prémio. Exploration
episodios_totais, penalidades_totais = 0, 0 

episodios = 20000

for i in range(episodios):
    feito = False # verifica se o treinamento terminou
    penalidade, premiacao = 0, 0
    estado = env.reset()

    while not feito:
        if random.uniform(0,1) < epsilon: # decidindo se será tomado uma ação aleatória ou se seguirá a política da tabela-q
            acao = env.action_space.sample()
        else:
            acao = np.argmax(tabela_q[estado])

        proximo_estado, premiacao, feito, info = env.step(acao) # As variáveis recebem a ação executada

        valor = tabela_q[estado, acao]
        maior_valor = np.max(tabela_q[proximo_estado])

        novo_valor = (1 - alpha) * valor + alpha * (premiacao + gamma * maior_valor) # atualizando o valor de q a partir da equação de Bellman
        tabela_q[estado, acao] = novo_valor # colocando este valor na tabela-q

        if premiacao == -10: # contabilizando os embarques/desembarques errados
            penalidade += 1

        episodios += 1
        estado = proximo_estado

        if i % 100 == 0:
            print(f"Episódio: {i}")
            clear_output(wait=True)
        else:
            exibir(acao, proximo_estado, premiacao, penalidades_totais, episodios, info)

    penalidades_totais += penalidade
    episodios_totais += episodios

print("Treinamento finalizado.")

print(f"Resutados depois de {episodios} episodios:")
print(f"Média de passos por episódio: {episodios_totais / episodios}")
print(f"Média de penalidades por episódio: {penalidades_totais / episodios}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
2 401 -1 13 603 {'prob': 1.0, 'TimeLimit.truncated': True} 
Treinamento finalizado.
Resutados depois de 603 episodios:
Média de passos por episódio: 2.0049751243781095
Média de penalidades por episódio: 0.03814262023217247


# Salva as informações da Tabela_q

In [ ]:
np.save("arquivo_tabela_q", tabela_q)
tabela_q = np.load("arquivo_tabela_q.npy")